In [170]:
import cv2
import numpy as np
import scipy.io

#        8   12  16  20
#        |   |   |   |
#        7   11  15  19
#    4   |   |   |   |
#    |   6   10  14  18
#    3   |   |   |   |
#    |   5---9---13--17
#    2    \         /
#     \    \       /
#      1    \     /
#       \    \   /
#        ------0-
connections = [
    (0, 1), (1, 2), (2, 3), (3, 4),
    (5, 6), (6, 7), (7, 8),
    (9, 10), (10, 11), (11, 12),
    (13, 14), (14, 15), (15, 16),
    (17, 18), (18, 19), (19, 20),
    (0, 5), (5, 9), (9, 13), (13, 17), (0, 17)
]

mat = scipy.io.loadmat("keypoint_gestures.mat")
keypoints = mat["keypoints"]
labels = mat["labels"]

def visualize(points, resolution=(480, 640, 3)):
    img = np.zeros(resolution)
    for point in points:
        x, y = point[0], point[1]
        cv2.circle(img, (int(x) * 2, int(y) * 2), 4, (255, 0, 0), 2)
    for connection in connections:
        x0, y0 = points[connection[0]]
        x1, y1 = points[connection[1]]
        cv2.line(img, (int(x0) * 2, int(y0) * 2), (int(x1) * 2, int(y1) * 2), (0, 255, 0), 2)
    cv2.imshow("Key Points", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [192]:
print(labels[0][20005])
visualize(keypoints[20005])

6


In [9]:
len(mat['keypoints'])
len(labels[0])

29282

# Process Data

In [45]:
import pandas as pd

In [96]:
X = mat['keypoints']
columns = np.array([('X' + str(x), 'Y' + str(x)) for x in range(1, 22)]).flatten()
x, y, z = np.shape(X)
X = X.reshape((x, y * z))
df = pd.DataFrame(data=X, columns=columns)
df['gesture'] = labels[0]
X_columns = ['X' + str(x) for x in range(1, 22)]
Y_columns = ['Y' + str(x) for x in range(1, 22)]
df[X_columns] = df[X_columns].divide(640.)
df[Y_columns] = df[Y_columns].divide(480.)
len(df)

29282

In [148]:
df[X_columns] = df[X_columns].divide(df['X1'], axis=0)
df[Y_columns] = df[Y_columns].divide(df['Y1'], axis=0)
df

,X1,Y1,X2,Y2,X3,Y3,X4,Y4,X5,Y5,...,Y17,X18,Y18,X19,Y19,X20,Y20,X21,Y21,gesture
0,1.0,1.0,0.843452,1.056137,0.724994,1.022185,0.655566,0.962955,0.636546,0.908738,...,0.755357,0.936965,0.733445,0.837969,0.706624,0.814078,0.737414,0.829680,0.757632,10
1,1.0,1.0,1.177046,1.035117,1.323947,0.966278,1.437599,0.923808,1.418797,0.881190,...,0.757993,1.083370,0.741576,1.203497,0.681920,1.219787,0.703975,1.190467,0.736948,10
2,1.0,1.0,0.925139,1.055485,0.837637,1.031221,0.767064,0.990368,0.732984,0.976571,...,0.876054,0.925816,0.785508,0.903861,0.752172,0.908464,0.786282,0.913025,0.819413,10
3,1.0,1.0,1.204531,1.039293,1.338035,0.950360,1.369972,0.830458,1.317565,0.762205,...,0.628515,0.917335,0.627954,0.974671,0.549243,0.964692,0.588689,0.919193,0.651605,10
4,1.0,1.0,0.897319,1.022892,0.818032,0.950411,0.776316,0.876225,0.775818,0.793361,...,0.722964,1.025603,0.533984,1.009484,0.551104,0.988913,0.659838,0.984016,0.726956,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29277,1.0,1.0,1.174264,1.004059,1.251325,0.925298,1.341532,0.856088,1.414291,0.817059,...,0.685761,1.285229,0.679497,1.371935,0.636845,1.430519,0.645944,1.433506,0.654495,9
29278,1.0,1.0,0.872181,0.946869,0.776622,0.890471,0.709013,0.846028,0.673392,0.805228,...,0.818671,0.846250,0.791844,0.766137,0.802504,0.759358,0.850233,0.765500,0.887564,9
29279,1.0,1.0,1.015619,0.921436,1.069154,0.791041,1.178922,0.687218,1.247048,0.614879,...,0.850613,1.363259,0.849446,1.424720,0.843843,1.357913,0.874500,1.293386,0.889737,9
29280,1.0,1.0,1.436349,0.909830,1.643644,0.801108,1.754931,0.693582,1.818784,0.632808,...,0.531989,1.062211,0.608499,1.157938,0.459205,1.215993,0.463516,1.213602,0.502210,9


## Test train split

In [100]:
from sklearn.model_selection import train_test_split

In [151]:
# Train, test sets split
X_train, X_test, y_train, y_test = train_test_split(df[columns], df['gesture'], test_size=0.25, random_state=42)

# Random Forest

In [108]:
from sklearn.ensemble import RandomForestClassifier
from matplotlib.legend_handler import HandlerLine2D
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

### Tuning hyperparameters using 10-fold CV

In [117]:
# Preparing of a list of n_estimators values to tune
n_estimators = [1, 2, 4, 8, 16, 32, 64, 100, 200]
# Prepare a list of depth values
depths = np.linspace(1, 30, 10, endpoint=True)

params_grid = {
    'n_estimators': n_estimators,
    'max_depth': depths
}

In [118]:
# Initialize a random forest classifier
rf_clf = RandomForestClassifier()
clf = GridSearchCV(rf_clf, params_grid, cv=10, verbose=10)

In [119]:
clf.fit(X_train, y_train)

Fitting 10 folds for each of 90 candidates, totalling 900 fits
[CV] max_depth=1.0, n_estimators=1 ...................................
[CV] ....... max_depth=1.0, n_estimators=1, score=0.146, total=   0.0s
[CV] max_depth=1.0, n_estimators=1 ...................................
[CV] ....... max_depth=1.0, n_estimators=1, score=0.133, total=   0.0s
[CV] max_depth=1.0, n_estimators=1 ...................................
[CV] ....... max_depth=1.0, n_estimators=1, score=0.154, total=   0.0s
[CV] max_depth=1.0, n_estimators=1 ...................................
[CV] ....... max_depth=1.0, n_estimators=1, score=0.145, total=   0.0s
[CV] max_depth=1.0, n_estimators=1 ...................................
[CV] ....... max_depth=1.0, n_estimators=1, score=0.138, total=   0.0s
[CV] max_depth=1.0, n_estimators=1 ...................................
[CV] ....... max_depth=1.0, n_estimators=1, score=0.143, total=   0.0s
[CV] max_depth=1.0, n_estimators=1 ...................................
[CV] ....... m

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.1s remaining:    0.0s


[CV] ....... max_depth=1.0, n_estimators=1, score=0.133, total=   0.0s
[CV] max_depth=1.0, n_estimators=1 ...................................
[CV] ....... max_depth=1.0, n_estimators=1, score=0.149, total=   0.0s
[CV] max_depth=1.0, n_estimators=2 ...................................
[CV] ....... max_depth=1.0, n_estimators=2, score=0.148, total=   0.0s
[CV] max_depth=1.0, n_estimators=2 ...................................
[CV] ....... max_depth=1.0, n_estimators=2, score=0.178, total=   0.0s
[CV] max_depth=1.0, n_estimators=2 ...................................
[CV] ....... max_depth=1.0, n_estimators=2, score=0.199, total=   0.0s
[CV] max_depth=1.0, n_estimators=2 ...................................
[CV] ....... max_depth=1.0, n_estimators=2, score=0.184, total=   0.0s
[CV] max_depth=1.0, n_estimators=2 ...................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.1s remaining:    0.0s


[CV] ....... max_depth=1.0, n_estimators=2, score=0.165, total=   0.0s
[CV] max_depth=1.0, n_estimators=2 ...................................
[CV] ....... max_depth=1.0, n_estimators=2, score=0.171, total=   0.0s
[CV] max_depth=1.0, n_estimators=2 ...................................
[CV] ....... max_depth=1.0, n_estimators=2, score=0.204, total=   0.0s
[CV] max_depth=1.0, n_estimators=2 ...................................
[CV] ....... max_depth=1.0, n_estimators=2, score=0.160, total=   0.0s
[CV] max_depth=1.0, n_estimators=2 ...................................
[CV] ....... max_depth=1.0, n_estimators=2, score=0.182, total=   0.0s
[CV] max_depth=1.0, n_estimators=2 ...................................
[CV] ....... max_depth=1.0, n_estimators=2, score=0.177, total=   0.0s
[CV] max_depth=1.0, n_estimators=4 ...................................
[CV] ....... max_depth=1.0, n_estimators=4, score=0.183, total=   0.1s
[CV] max_depth=1.0, n_estimators=4 ...................................
[CV] .

...

[CV]  max_depth=26.77777777777778, n_estimators=2, score=0.674, total=   0.3s
[CV] max_depth=26.77777777777778, n_estimators=2 .....................
[CV]  max_depth=26.77777777777778, n_estimators=2, score=0.679, total=   0.2s
[CV] max_depth=26.77777777777778, n_estimators=2 .....................
[CV]  max_depth=26.77777777777778, n_estimators=2, score=0.672, total=   0.2s
[CV] max_depth=26.77777777777778, n_estimators=2 .....................
[CV]  max_depth=26.77777777777778, n_estimators=2, score=0.679, total=   0.3s
[CV] max_depth=26.77777777777778, n_estimators=2 .....................
[CV]  max_depth=26.77777777777778, n_estimators=2, score=0.660, total=   0.2s
[CV] max_depth=26.77777777777778, n_estimators=4 .....................
[CV]  max_depth=26.77777777777778, n_estimators=4, score=0.791, total=   0.6s
[CV] max_depth=26.77777777777778, n_estimators=4 .....................
[CV]  max_depth=26.77777777777778, n_estimators=4, score=0.770, total=   0.7s
[CV] max_depth=26.7777777777

[CV]  max_depth=26.77777777777778, n_estimators=100, score=0.879, total=  10.0s
[CV] max_depth=26.77777777777778, n_estimators=100 ...................
[CV]  max_depth=26.77777777777778, n_estimators=100, score=0.868, total=  10.0s
[CV] max_depth=26.77777777777778, n_estimators=100 ...................
[CV]  max_depth=26.77777777777778, n_estimators=100, score=0.872, total=   9.9s
[CV] max_depth=26.77777777777778, n_estimators=100 ...................
[CV]  max_depth=26.77777777777778, n_estimators=100, score=0.869, total=  10.0s
[CV] max_depth=26.77777777777778, n_estimators=100 ...................
[CV]  max_depth=26.77777777777778, n_estimators=100, score=0.872, total=  10.0s
[CV] max_depth=26.77777777777778, n_estimators=100 ...................
[CV]  max_depth=26.77777777777778, n_estimators=100, score=0.867, total=  10.0s
[CV] max_depth=26.77777777777778, n_estimators=100 ...................
[CV]  max_depth=26.77777777777778, n_estimators=100, score=0.883, total=  10.0s
[CV] max_depth

[CV] ...... max_depth=30.0, n_estimators=8, score=0.811, total=   0.8s
[CV] max_depth=30.0, n_estimators=8 ..................................
[CV] ...... max_depth=30.0, n_estimators=8, score=0.821, total=   0.8s
[CV] max_depth=30.0, n_estimators=8 ..................................
[CV] ...... max_depth=30.0, n_estimators=8, score=0.820, total=   0.9s
[CV] max_depth=30.0, n_estimators=16 .................................
[CV] ..... max_depth=30.0, n_estimators=16, score=0.857, total=   1.8s
[CV] max_depth=30.0, n_estimators=16 .................................
[CV] ..... max_depth=30.0, n_estimators=16, score=0.848, total=   1.6s
[CV] max_depth=30.0, n_estimators=16 .................................
[CV] ..... max_depth=30.0, n_estimators=16, score=0.856, total=   1.7s
[CV] max_depth=30.0, n_estimators=16 .................................
[CV] ..... max_depth=30.0, n_estimators=16, score=0.855, total=   1.6s
[CV] max_depth=30.0, n_estimators=16 .................................
[CV] .

[Parallel(n_jobs=1)]: Done 900 out of 900 | elapsed: 57.0min finished


GridSearchCV(cv=10, estimator=RandomForestClassifier(),
             param_grid={'max_depth': array([ 1.        ,  4.22222222,  7.44444444, 10.66666667, 13.88888889,
       17.11111111, 20.33333333, 23.55555556, 26.77777778, 30.        ]),
                         'n_estimators': [1, 2, 4, 8, 16, 32, 64, 100, 200]},
             verbose=10)

In [122]:
model = clf.best_estimator_

In [152]:
model.fit(X_train, y_train)

RandomForestClassifier(max_depth=26.77777777777778, n_estimators=200)

In [153]:
model.score(X_test, y_test)

0.8847152028411419

In [154]:
import pickle

filename = 'rand_forest.sav'
pickle.dump(model, open(filename, 'wb'))

In [155]:
X_test

,X1,Y1,X2,Y2,X3,Y3,X4,Y4,X5,Y5,...,X17,Y17,X18,Y18,X19,Y19,X20,Y20,X21,Y21
17735,1.0,1.0,1.271608,1.005560,1.507063,0.915980,1.682156,0.825612,1.813272,0.751146,...,1.164738,0.331759,0.899969,0.708283,0.919985,0.575584,0.956051,0.495125,0.985244,0.429757
7739,1.0,1.0,1.625026,0.960984,1.894031,0.857778,1.984351,0.749398,1.914319,0.663270,...,0.865897,0.686726,0.548832,0.685685,0.571818,0.632290,0.606098,0.653147,0.542712,0.692670
28239,1.0,1.0,1.030886,0.880019,1.147578,0.730807,1.316125,0.651886,1.421949,0.618982,...,1.405263,0.848717,1.426639,0.975557,1.573023,0.956007,1.525511,0.969887,1.445920,0.959930
17166,1.0,1.0,0.896804,1.001125,0.819538,0.957726,0.753803,0.912469,0.688988,0.878863,...,0.955026,0.454233,1.041232,0.758892,1.070895,0.670703,1.087331,0.603870,1.098288,0.542568
25741,1.0,1.0,0.875343,0.802928,0.877240,0.595890,0.892676,0.417128,0.807746,0.304744,...,1.377896,0.798658,1.526346,0.879377,1.737233,0.810161,1.588838,0.836733,1.473719,0.852598
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10867,1.0,1.0,0.830502,0.937545,0.827212,0.814269,0.902253,0.732988,0.997295,0.695129,...,1.087996,0.581396,1.158642,0.712299,1.135250,0.589018,1.101692,0.629950,1.074503,0.686930
21040,1.0,1.0,0.897073,1.049729,0.819176,1.000975,0.762474,0.921705,0.713814,0.868642,...,0.948580,0.819345,1.016248,0.738030,1.026924,0.691076,1.029179,0.698293,1.028687,0.703918
28108,1.0,1.0,0.932667,0.960358,0.863946,0.876340,0.807150,0.809187,0.742099,0.735564,...,0.781705,0.706105,0.820385,0.610795,0.755916,0.605453,0.741831,0.661227,0.750206,0.676787
7690,1.0,1.0,1.195762,1.027173,1.335196,0.941340,1.411743,0.805464,1.427168,0.687022,...,1.014680,0.808750,0.897197,0.790496,0.861641,0.745255,0.892994,0.767471,0.888229,0.816981


In [156]:
loaded_model = pickle.load(open(filename, 'rb'))

In [157]:
loaded_model.score(X_test, y_test)

0.8847152028411419

In [199]:
loaded_model.predict_proba(X_test[:1])

array([[0.   , 0.   , 0.   , 0.01 , 0.015, 0.965, 0.005, 0.   , 0.005,
        0.   , 0.   ]])

In [205]:
visualize(keypoints[17735])